# Code for Pulling Lyrics from Genius

In [1]:
from lyricsgenius import Genius
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import sqlite3
conn = sqlite3.connect('../datasets/lyrics.db')
from requests.exceptions import Timeout

In [2]:
#Lyrcis
genius = Genius(os.getenv('GENIUS_KEY'))
genius.timeout = 15
genius.remove_section_headers = True

#need to write function to generate for each and store in a datastructure to not duplicate searches.

In [27]:
#final_df = pd.read_csv("../datasets/Final database.csv")
full_df = pd.read_csv("../datasets/Database to calculate popularity.csv")


In [57]:
lyric_df = pd.read_sql_query("SELECT * FROM lyrics", conn)


In [58]:
holder_df = full_df[full_df['country']=='USA']
#remove nan and remove uri that are already in lyric_df
holder_df = holder_df[~holder_df['uri'].isin(lyric_df['uri'])]
#remove nan in holder_df
holder_df = holder_df.dropna()
#remove duplicate uri in holder_df
holder_df = holder_df.drop_duplicates(subset='uri')
print(len(holder_df['uri'].unique()))
print(len(holder_df))
holder_df


43
43


,Unnamed: 0,country,date,position,uri,track,title,artist
281193,281193,USA,01/11/2020,194.0,https://open.spotify.com/track/1TfqLAPs4K3s2rJ...,"Sweet Dreams - Remastered , Eurythmics, Anni...",Sweet Dreams - Remastered,"Eurythmics, Annie Lennox, Dave Stewart"
281302,281302,USA,31/10/2020,103.0,https://open.spotify.com/track/0kwuKfWntoGh0EW...,"Stranger Things , Kyle Dixon & Michael Stein",Stranger Things,Kyle Dixon & Michael Stein
285389,285389,USA,11/10/2020,190.0,https://open.spotify.com/track/31iqpzg1ZXLLgq0...,"Throat Ba, , BRS Kash",Throat Ba,", BRS Kash"
288794,288794,USA,24/09/2020,195.0,https://open.spotify.com/track/5FmvaZGd6fulojS...,"One Beer , HIXTAPE",One Beer,HIXTAPE
289328,289328,USA,21/09/2020,129.0,https://open.spotify.com/track/5CcPj2miBzqHAxm...,"Thinkin of You , Chan Se Park",Thinkin of You,Chan Se Park
289329,289329,USA,21/09/2020,130.0,https://open.spotify.com/track/1V8bPfh3pgdsHU5...,"Blurred Lines , Oda Kirkwood",Blurred Lines,Oda Kirkwood
289340,289340,USA,21/09/2020,141.0,https://open.spotify.com/track/38HY8JklUp2o1hc...,"Miser , WAVYVIBE",Miser,WAVYVIBE
289344,289344,USA,21/09/2020,145.0,https://open.spotify.com/track/7icOlPlKJ3JSuZU...,"Love Is So Sweet , Missing Seoul",Love Is So Sweet,Missing Seoul
289435,289435,USA,20/09/2020,36.0,https://open.spotify.com/track/6F36SoTLd7pky1n...,"Longing Heart , Mayar Jenisk",Longing Heart,Mayar Jenisk
289674,289674,USA,19/09/2020,75.0,https://open.spotify.com/track/7nz7N7kYxYAir3m...,"Without You , Jennifer Ormond",Without You,Jennifer Ormond


In [46]:
#to make searches faster, we can group by artist and then create a list of tuples of the the title and the uri
new_df = holder_df.groupby('artist').agg({'title':'first','uri':'first'})
#temp_df = holder_df.groupby('artist')['title'].apply(list).reset_index()
new_df



,title,uri
artist,,
", BRS Kash",Throat Ba,https://open.spotify.com/track/31iqpzg1ZXLLgq0...
", Kylie Minogue",Santa Ba,https://open.spotify.com/track/2SBr3MK1sdMs8Ix...
", YoungBoy Never Broke Again",I-10 Ba,https://open.spotify.com/track/2W9uAM0aMsZVxf3...
A Boogie Wit da Hoodie,Timeless,https://open.spotify.com/track/0q5DrtpnnGpOvBy...
A R I Z O N A,Oceans Away,https://open.spotify.com/track/6A8dnC0xkiuWN4B...
...,...,...
"Zay Hilfigerrr, Zayion McCall",Juju on That Beat,https://open.spotify.com/track/1lItf5ZXJc1by9S...
alt-J,In Cold Blood,https://open.spotify.com/track/0jeJR3LdrC2QJNv...
"i love u , gnash",i hate u,https://open.spotify.com/track/7vRriwrloYVaoAe...


In [53]:
def storeLyrics(songName,artistName,uri,cursor): #sep and local
    artist = None
    for artist_i in artistName.split('&'):
        artist = genius.search_artist(artist_i, max_songs=0)
        if artist != None:
            break
    if artist==None:
        return None
    song = None
    for song_i in songName.split('-'):
        song = genius.search_song(song_i, artist.name)
        if song != None:
            break
    if song==None:
        return None
    lyrics = song.lyrics.replace('(','').replace(')','')
    query = 'INSERT INTO lyrics(song,artist,lyric,uri) VALUES (\"'+str(songName).replace('\"','\'')+'\",\"'+str(artistName.split(',')).replace('\"','\'')+'\",\"'+lyrics.replace('\"','\'')+'\",\''+str(uri).replace('\"','\'')+'\')'
    cursor.execute(query)
    conn.commit()
def test_store(songName,artistName):
    artist = genius.search_artist(artistName, max_songs=0)
    if artist==None:
        return None
    song = genius.search_song(songName, artist.name)
    if song==None:
        return None
    lyrics = song.lyrics.replace('(','').replace(')','')

In [56]:
cur = conn.cursor()
#cur.execute("CREATE TABLE lyrics (song text NOT NULL, artist text NOT NULL, lyric text NOT NULL, uri text NOT NULL);")
taken = []
for index,row in holder_df[::-1].iterrows():
    retries = 0
    if (row['uri'] in taken):
        pass
    else:
        while retries<3:
            try:
                storeLyrics(row['title'],row['artist'],row['uri'],cur)
                taken.append(row['uri'])
                retries = 3
            except Timeout as e:
                retries += 1
                continue

#conn.close()

Searching for songs by  The Rolling Stones ...

Changing artist name to 'The Rolling Stones'
Done. Found 0 songs.
Searching for "  Satisfaction " by The Rolling Stones...
Done.
Searching for songs by  Bee Gees ...

Changing artist name to 'Bee Gees'
Done. Found 0 songs.
Searching for " Stayin' Alive " by Bee Gees...
Done.
Searching for songs by  Pharrell Williams ...

Changing artist name to 'Pharrell Williams'
Done. Found 0 songs.
Searching for " Happy " by Pharrell Williams...
Done.
Searching for songs by  Axwell /\ Ingrosso, Kid Ink ...

No results found for ' Axwell /\ Ingrosso, Kid Ink '.
Searching for songs by  Migos ...

Changing artist name to 'Migos'
Done. Found 0 songs.
Searching for " T" by Migos...
Done.
Searching for songs by  Oasis ...

Changing artist name to 'Oasis'
Done. Found 0 songs.
Searching for " Wonderwall " by Oasis...
Done.
Searching for songs by  Nick Jonas, Nicki Minaj ...

Changing artist name to 'Nick Jonas & Nicki Minaj'
Done. Found 0 songs.
Searching for 

In [ ]:
conn.commit()
conn.close()